# Test #1 - The influence of geology, faults and fractures

## Initialization

In [1]:
# Load pyKasso and other useful packages
import copy
import numpy as np
import pykasso as pk
app = pk.pykasso()

In [2]:
# Create new project and set project settings
project_path = 'geometry_01'
grid_parameters = {
    'x0' : 0,
    'y0' : 0,
    'z0' : 50,
    'nx' : 100,
    'ny' : 50,
    'nz' : 50,
    'dx' : 10,
    'dy' : 10,
    'dz' : 10
}
app.new_project(grid_parameters=grid_parameters, project_name=project_path)

In [3]:
# Construct the model parameters dictionary
model_parameters = {
    'sks' : {
        'seed' : 1111,
    },
    'outlets' : {
        'number'     : 1,
        'data'       : [[994, 250]],
        'importance' : [1],
        'mode'       : 'domain_bottom',
    },
    'inlets' : {
        'number'     : 1,
        'data'       : [[-5, 250]],
        'per_outlet' : [1],
        'importance' : [1],
        'mode'       : 'domain_surface',
    },
}

## Easiest case : No domain delimitations + no geological features 

In [4]:
# Compute the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    model_parameters['sks']['algorithm'] = algorithm
    app.model.generate(model_parameters)

A


In [6]:
app.model.conceptual_model
app.model.conceptual_model_table

,feature,id-feature,cost
id,,,
0,Out,NaN,10.0
100,Geology,1.0,0.4


In [5]:
# Visualize the results
settings = {
    'ghost_values': [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
app.visualizer.pv_show([0, 1], ['karst'], settings=settings)

## Influence of the faults

In [6]:
# Construct the fault
faults = np.full((100,50,50), 0)
faults[:, :, 25] = 1

# Visualize the fault
# app.visualizer.show_array(faults)

# Update the model parameters dictionary
new_param = copy.deepcopy(model_parameters)
new_param['faults'] = {'data' : faults}

In [7]:
# Compute the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_param['sks']['algorithm'] = algorithm
    app.model.generate(new_param)

A


In [8]:
# Visualizes the results
settings = {
    'ghost_values' : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
app.visualizer.pv_show([2, 3], ['karst', 'faults'], settings=settings)

## Influence of the fracturation

In [9]:
# Declare the fracturation settings
fractures = {
    'seed': 111111,
    'settings': {
        'family_01': {
            'alpha'        : 2,
            'density'      : 0.00005,
            'orientation'  : [90, 120],
            'dip'          : 90,
            'length'       : [300, 500],
        }, 
        'family_02': {
            'alpha'        : 2,
            'density'      : 0.00005,
            'orientation'  : 0,
            'dip'          : [0, 90],
            'length'       : [400,600],
        }
    }
}

# Updates the settings
new_param = copy.deepcopy(model_parameters)
new_param['fractures'] = fractures

In [10]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_param['sks']['algorithm'] = algorithm
    app.model.generate(new_param)

A


In [11]:
# Access the list of generated fractures
app.model.fractures.fractures
# app.model.fractures.families

,family_id,x,y,z,radius,orientation,dip,normal
0,1,1085.616879,699.204133,758.114736,226.302985,114.969195,90.000000,"(-0.4221309281017555, -0.9065348749716972, -6...."
1,1,580.889012,332.543086,514.839089,230.738441,101.949688,90.000000,"(-0.20705269035092258, -0.9783297927684943, -6..."
2,1,356.332719,532.351925,746.243942,152.860164,116.143092,90.000000,"(-0.4406144449613087, -0.897696446963804, -6.1..."
3,1,172.467134,-22.855182,-103.865691,192.315086,103.267445,90.000000,"(-0.22949675474698863, -0.9733094264213208, -6..."
4,1,1028.131196,531.745430,-204.287175,159.220691,96.492085,90.000000,"(-0.11306595675047629, -0.9935874845347537, -6..."
...,...,...,...,...,...,...,...,...
67,2,271.546046,689.074839,522.725254,216.367564,0.000000,47.042971,"(0.7318649852901103, -4.481380558217792e-17, -..."
68,2,930.107632,-182.253714,670.177368,280.714941,0.000000,28.194924,"(0.47247268242020196, -2.893060791052321e-17, ..."
69,2,-212.820709,-213.541952,702.756662,253.073117,0.000000,65.585087,"(0.9105761064040493, -5.575670570438893e-17, -..."
70,2,326.762613,138.003454,-18.304914,275.977987,0.000000,87.761368,"(0.999236805757383, -6.118560778805023e-17, -0..."


In [12]:
# Visualize the results
settings = {
    'ghost_values' : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
app.visualizer.pv_show([4, 5], ['karst', 'fractures'], settings=settings)

## Influence of the geology

In [13]:
# Construct the geology
geology = np.full((100,50,50), 1)
geology[:, 35:50, :] = 2

# Visualize the geology
# app.visualizer.show_array(geology)

# Update the parameters
new_param = copy.deepcopy(model_parameters)
new_param['geology'] = {
    'data' : geology,
    'costs' : {1: 0.6, 2: 0.4}
}

In [14]:
# Compute the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_param['sks']['algorithm'] = algorithm
    app.model.generate(new_param)

A


In [15]:
# Visualize the results
settings = {
    'ghost_values'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
    'colorbar': True
}
app.visualizer.pv_show([6,7], ['karst', 'cost'], settings=settings)